In [ ]:
# TODO: REMOVE ME!
from plaster.tools.ipynb_helpers.displays import restart_kernel; restart_kernel()

In [ ]:
import os
os.environ["MPLCONFIGDIR"] = "/tmp"

In [1]:
import cv2
import itertools
import numpy as np
import os
import pandas as pd
import random
from IPython.display import HTML, display
from plaster.run.calib.calib import RegPSF, approximate_psf
from plaster.run.job import JobResult
from plaster.run.plots import plots, plots_dev
from plaster.run.plots.plots_sigproc import plot_psfs, circle_locs, sigproc_v2_im, sigproc_v2_movie_from_df, sigproc_v2_im_from_df
from plaster.run.plots.plots_sigproc import wizard_xy_df, wizard_scat_df, wizard_raw_images
from plaster.run.run import RunResult
from plaster.run.sigproc_v2 import sigproc_v2_worker as worker
from plaster.run.sigproc_v2.sigproc_v2_result import df_filter, radmat_from_df_filter, df_to_radmat, mean_non_dark_asr
from plaster.run.sigproc_v2.synth import Synth
from plaster.run.sigproc_v2 import bg
from plaster.run.features import features
from plaster.tools.image import imops
from plaster.tools.image.coord import WH, XY, roi_shift, clip2d
from plaster.tools.ipynb_helpers.displays import hd, movie, md
from plaster.tools.log.log import error, debug
from plaster.tools.schema import check
from plaster.tools.utils import data
from plaster.tools.utils import utils
from plaster.tools.zap import zap
from plaster.tools.zplots import zplots
from plaster.tools.ipynb_helpers import displays
z = zplots.setup()

Loading BokehJS ...

In [2]:
# REMOVE ME
from plaster.run.base_result import disable_disk_memoize_from_notebook
disable_disk_memoize_from_notebook()

In [3]:
job = JobResult.from_context(dev_override="/erisyon/internal/jobs_folder/abbe7_1t")
run = job.sigproc_v2

In [4]:
# Peak run-length distribution
# Number of peaks that fall within n-stds of dark by cycle x.

# Radiometry heatmaps
# Sorted by run_len then by mean_brightness
# Sorted by hierarchical clustering
# Sorted by field then run_len then mean_brightness
# Per channel


In [5]:
mdf = features(run)
mask = mdf.run_len < run.sigproc_v2.n_cycles
mdf = mdf[mask]
cols = ["field_i", "aln_y", "aln_x", "flowcell_y", "flowcell_x", "radius", "run_len", "run_med", "run_iqr", "nei_med", "nei_iqr", "sig_med", "sig_iqr", "asr_cy0"]
# TODO: Add subsampling
with z(_cols=len(cols), _notools=True, _noaxes=True, _size=80, alpha=0.05):
    for yi, col_y in enumerate(cols):
        for xi, col_x in enumerate(cols):
            f_title = col_x if yi == 0 else ""
            f_y_axis_label = col_y if xi == 0 else ""
            z.scat(x=mdf[col_x], y=mdf[col_y], f_title=f_title, f_y_axis_label=f_y_axis_label)

AttributeError: 

In [ ]:
hd("h1", f"Calibration")
if "calibration" not in run.sigproc_v2.params:
    print("No calibration was provded to this run")
else:
    for ch_i in range(run.ims_import.n_channels):
        hd("h2", f"Channel {ch_i}")

        with z(_cols=3, _size=250):
            reg_psf = run.sigproc_v2.params.calibration.reg_psf(ch_i=ch_i)
            check.t(reg_psf, RegPSF)
            psf_ims = reg_psf.render()
            plot_psfs(psf_ims, scale=3.0, f_title=f"Regional PSF", _noaxes=True, _notools=True, _zplots_context=z)

            z.cols(
                np.max(reg_psf.params[ch_i][:, :, 0:2], axis=2).flatten(),
                f_x_axis_label="Region #",
                f_y_axis_label="peak size",
                f_title="Regional PSF peak size",
            )

            illum = run.sigproc_v2.params.calibration.reg_illum().interp(ch_i)
            z.im(1.0 / illum, f_title="Regional Illumination Balance Map", _cspan=(0, 1.0))


In [ ]:
# TODO: Add a "recavlration" for this experiment. If this experiment had been used as a calibration

In [ ]:
hd("h1", f"Signal distributions")
n_peaks_per_field = run.sigproc_v2.peaks().groupby("field_i").count().field_peak_i
n_peaks_max = np.max(n_peaks_per_field)
n_fields = run.sigproc_v2.n_fields
for ch_i in range(run.ims_import.n_channels):
    hd("h2", f"Channel {ch_i}")
    
    dark = run.sigproc_v2.dark_estimate(ch_i=ch_i)
    sig = run.sigproc_v2.sig()[:, ch_i, :]
    sig = sig.transpose(1, 0)
    beta_est = np.median(sig[sig > dark])
    sig_non_dark = np.where(sig > dark, sig, np.nan)

    sig_non_dark_by_field = np.full((n_fields, n_peaks_max), np.nan)
    for fl_i in range(n_fields):
        _sig = run.sigproc_v2.sig(fields=fl_i)[:, ch_i, 0]
        _sig_non_dark = np.where(_sig > dark, _sig, np.nan)
        sig_non_dark_by_field[fl_i, 0:len(_sig_non_dark)] = _sig_non_dark

    with z(_cols=3):
        z.hist(
            sig,
            _vertical=beta_est,
            f_x_axis_label="Signal",
            f_title="Signal distributions, all cycles, all fields"
        )
        
        z.hist(
            sig[0, :],
            _vertical=beta_est,
            f_x_axis_label="Signal",
            f_title="Signal distributions, @ first cycle, all fields"
        )
        
        z.hist(
            sig[-1, :],
            _vertical=beta_est,
            f_x_axis_label="Signal",
            f_title="Signal distributions, @ last cycle, all fields"
        )
        
        z.distr(
            sig_non_dark,
            _vertical=beta_est,
            _percentiles=(0, 25, 50, 75, 99),
            _nogrid=True,
            f_x_axis_label="Signal", f_y_axis_label="Cycle",
            f_title="Non-dark signal by cycle, all fields"
        )

        z.distr(
            sig_non_dark_by_field,
            _vertical=beta_est,
            _percentiles=(0, 25, 50, 75, 99),
            _nogrid=True,
            f_x_axis_label="Signal", f_y_axis_label="Field",
            f_title="Non-dark signal @ cycle=0, by field"
        )
    

In [ ]:
hd("h1", f"Aspect ratios @ cycle=0, max all channels (a proxy for collisions)")
asr_thresh = run.sigproc_v2.asr_threshold()

asr = run.sigproc_v2.aspect_ratio()[:, :, 0]
n_peaks = asr.shape[0]
asr = np.max(asr, axis=1)
n_keep = (asr < run.sigproc_v2.asr_threshold()).sum()

hd("h2", f"Keep {100 * n_keep / n_peaks:.1f}%, Reject {100 * (n_peaks-n_keep) / n_peaks:.1f}%")

with z(_merge=True):
    bins = np.linspace(1, 3, 200)
    _h, _ = np.histogram(asr, bins=bins)
    z.hist(asr, _bins=bins)
    z.line(x=[asr_thresh, asr_thresh], y=[0, np.max(_h)], color="red")

In [ ]:
hd("h1", f"Peak density")

n_peaks_per_field = run.sigproc_v2.peaks().groupby("field_i").count().field_peak_i
hd("h2", f"<i>Mean pixels per peak:</i> {run.ims_import.dim**2 / np.mean(n_peaks_per_field):.1f}")

field_df = run.ims_import.metadata()[["field_i", "stage_x", "stage_y"]].groupby("field_i").mean().join(n_peaks_per_field)
field_df["size"] = field_df.field_peak_i * 0.03

with z(_cols=2):
    z.cols(
        n_peaks_per_field,
        f_title="# peaks per field",
        f_x_axis_label="field",
        f_y_axis_label="n_peaks",
    )
    z.scat(
        source=field_df, x="stage_x", y="stage_y", size="size",
        f_title="# peaks per field in stage X/Y coords.",
        f_x_axis_label="stage x",
        f_y_axis_label="stage y",
    )


In [ ]:
qdf = run.ims_import.qualities()
qdf = qdf[(qdf.field_i < run.sigproc_v2.n_fields) & (qdf.channel_i == ch_i)]
mean_qdf = qdf.groupby("field_i").mean().reset_index().sort_values("quality")
median_fl_i = int(mean_qdf.iloc[len(mean_qdf) // 2].field_i.astype(int))

hd("h1", f"All-cycles movie @ field={median_fl_i} (has median_quality)")

for ch_i in range(run.ims_import.n_channels):
    hd("h2", f"Channel {ch_i}")


    df = run.sigproc_v2.fields__n_peaks__peaks__radmat(fields=median_fl_i)
    dark = run.sigproc_v2.dark_estimate(ch_i=0, fields=median_fl_i)
    max_asr = run.sigproc_v2.asr_threshold()
    filt_df = df_filter(df, dark=dark, max_aspect_ratio=max_asr)
    sigproc_v2_movie_from_df(run, filt_df)

In [ ]:
hd("h1", f"All-fields movie @ cycle=0")

for ch_i in range(run.ims_import.n_channels):
    hd("h2", f"Channel {ch_i}")

    ims = np.zeros((run.sigproc_v2.n_fields, run.ims_import.dim, run.ims_import.dim))
    for fl_i in range(run.sigproc_v2.n_fields):
        im = run.sigproc_v2.aln_ims[fl_i, ch_i, 0]
        h, w = im.shape
        ims[fl_i, 0:h, 0:w] = im

    displays.movie(ims,
        _labels=[
            f"fl_i:{fl_i} ch_i:{ch_i} cy_i: 0"
            for fl_i in range(run.sigproc_v2.n_fields)
        ],
        _duration=200,
    )


In [ ]:
n_cycles = run.ims_import.n_cycles

hd("h1", f"Pixel Intensity Distributions (by cycle @ 5 random fields)")
md("The <span style='color:red'>red line</span> is set arbitrarily at 500 for easier job to job refernce.")

for ch_i in range(run.ims_import.n_channels):
    hd("h2", f"Channel {ch_i}")

    n_samples = 10000
    bg_samps = np.zeros((n_cycles, n_samples))
    fg_samps = np.zeros((n_cycles, n_samples))
    idx = np.arange(run.ims_import.dim**2)
    cy_labels = [f"cy {cy_i}" for cy_i in range(n_cycles)]
    for fl_i in utils.ispace(0, run.ims_import.n_fields, 5):
        for cy_i in range(n_cycles):
            im = run.ims_import.ims[fl_i, ch_i, cy_i]
            bg_im, fg_mask = bg.background_extract(im, approximate_psf(), dilate=0)
            _idx = data.subsample(idx[fg_mask.flatten()], n_samples)
            bg_samps[cy_i, :] = data.subsample(bg_im.flatten(), n_samples)
            fg_samps[cy_i, :] = im.flatten()[_idx]

    with z(_cols=3):
        z.distr(bg_samps, _vertical=500.0, _percentiles=(0, 25, 50, 75, 99), f_x_axis_label="Intensity", f_y_axis_label="Cycle", f_title="background pixels only")
        z.distr(fg_samps, _vertical=500.0, _percentiles=(0, 25, 50, 75, 99), f_x_axis_label="Intensity", f_y_axis_label="Cycle", f_title="foreground pixels only")
    
        bg_medians = np.nanmedian(bg_samps, axis=1)
        fg_medians = np.nanmedian(fg_samps, axis=1)
        z.scat(
            x=fg_medians,
            y=bg_medians,
            f_x_axis_label="FG Med. Inentsity", f_y_axis_label="BG Med. Inentsity",
            _label=cy_labels
        )
    

In [ ]:
hd("h1", f"Field Quality")
for ch_i in range(run.ims_import.n_channels):
    hd("h2", f"Channel {ch_i}")
    qdf = run.ims_import.qualities()
    qdf = qdf[(qdf.field_i < run.sigproc_v2.n_fields) & (qdf.channel_i == ch_i)].sort_values(["quality"])
    z.hist(qdf.quality, _size_x=800, _size_y=150, f_title=f"Quality distribution")

    row_iz = utils.ispace(0, len(qdf), 3)

    # COMBINE all images for common percentile calculations
    ims = np.concatenate([
        run.sigproc_v2.aln_ims[row.field_i, row.channel_i, row.cycle_i].flatten()
        for row in qdf.iloc[row_iz].itertuples()
    ])
    bot, top = np.percentile(ims, (50, 99.9))

    # SHOW example of worst, median, and best all using the same cspan
    hd("h3", f"Examples of frames by quality")
    with z(_cols=3, _cspan=(bot, top)):
        names = ("worst", "median", "best")
        for name, row in zip(names, qdf.iloc[row_iz].itertuples()):
            z.im(run.sigproc_v2.aln_ims[row.field_i, row.channel_i, row.cycle_i], f_title=f"{name} fl_i={row.field_i} cy_i={row.cycle_i}, qual={row.quality:.0f}")

In [ ]:
hd("h1", f"Field Alignment")
field_df = run.sigproc_v2.fields().copy()
field_df["alignment"] = np.sqrt(field_df.aln_x**2 + field_df.aln_y**2)
alignment = field_df.groupby("field_i").alignment.max().values
z.cols(alignment, f_x_axis_label="field_i", f_y_axis_label="n_pixels", f_title="Max. alignment distance by field")